### Likelihood-Free Parameter Inference on the MA2 Model

This notebook illustrates neural network-based (henceforth referred to as xNN) inference and approximate Bayesian computation.

The xNN models learn the relationship ${\bf y} \rightarrow {\bf \theta}$, where ${\bf y}$ is a time series response and ${\bf \theta}$ are the parameters of the descriptive model.

Therefore, we require a training set of the form $({\bf y, \theta})$ to train the xNN models. We will first generate such a training set.

In [1]:
# Imports
from ma2_model import simulate, prior
import numpy as np

In [2]:
# Set up simulation
sim = simulate
true_param = [0.6, 0.2]  # true \theta moving average2

data = simulate(true_param)

n = 10000
train_thetas = np.array(prior(n=n))
train_ts = np.expand_dims(np.array([simulate(p, n=100) for p in train_thetas]), 2)

validation_thetas = np.array(prior(n=10000))
validation_ts = np.expand_dims(np.array([simulate(p, n=100) for p in validation_thetas]), 2)

test_thetas = np.array(prior(n=10000))
test_ts = np.expand_dims(np.array([simulate(p, n=100) for p in validation_thetas]), 2)

abc_trial_thetas = np.array(prior(n=30000))
abc_trial_ts = np.expand_dims(np.array([simulate(p, n=100) for p in abc_trial_thetas]), 2)

In [3]:
# Set up training data in the right format
train_ts = train_ts.transpose((0, 2, 1))
validation_ts = validation_ts.transpose((0, 2, 1))
test_ts = test_ts.transpose((0, 2, 1))
data = data.reshape(1,1,100)

In [4]:
# Set input and output shape for the CNN
input_shape = (100,1)
output_shape = 2

In [5]:
# Set up the search space for inference
dmin = [-2, -1]
dmax = [4, 2]

In [6]:
# Routines to normalize and denormalize data
# Makes training easier
def normalize_data(data, dmin, dmax):
    dmin = np.array(dmin)
    dmax = np.array(dmax)
    return (data - dmin)/(dmax-dmin)

def denormalize_data(data, dmin, dmax):
    dmin = np.array(dmin)
    dmax = np.array(dmax)
    denorm = data * (dmax-dmin) + dmin
    return denorm

In [7]:
normed_thetas = normalize_data(train_thetas, dmin, dmax)
normed_thetas_val = normalize_data(validation_thetas, dmin, dmax)

In [8]:
# Import the xNN models
from sciope.models.cnn_regressor import CNNModel
from sciope.models.dnn_regressor import DNNModel
from sciope.models.pen_regressor import PENModel

In [9]:
# Instantiate the models
model_cnn = CNNModel(input_shape, output_shape)
model_dnn = DNNModel(input_shape, output_shape)
model_pen = PENModel(input_shape, output_shape, pen_nr=10)

In [10]:
history_cnn = model_cnn.train(train_ts, normed_thetas, batch_size=256, 
                      epochs=500, verbose=1, learning_rate=0.001, 
                      early_stopping_patience=5,
                      validation_inputs=validation_ts, validation_targets=normed_thetas_val)

Train on 10000 samples, validate on 10000 samples
Epoch 1/500
10000/10000 [==============================] - 9s 950us/sample - loss: 0.1366 - mae: 0.2427 - val_loss: 0.0835 - val_mae: 0.2556
Epoch 2/500
10000/10000 [==============================] - 5s 471us/sample - loss: 0.0105 - mae: 0.0798 - val_loss: 0.0744 - val_mae: 0.2409
Epoch 3/500
10000/10000 [==============================] - 6s 604us/sample - loss: 0.0069 - mae: 0.0650 - val_loss: 0.0553 - val_mae: 0.2002
Epoch 4/500
10000/10000 [==============================] - 5s 535us/sample - loss: 0.0071 - mae: 0.0653 - val_loss: 0.0326 - val_mae: 0.1500
Epoch 5/500
10000/10000 [==============================] - 4s 434us/sample - loss: 0.0050 - mae: 0.0556 - val_loss: 0.0298 - val_mae: 0.1443
Epoch 6/500
10000/10000 [==============================] - 6s 594us/sample - loss: 0.0047 - mae: 0.0535 - val_loss: 0.0248 - val_mae: 0.1357
Epoch 7/500
10000/10000 [==============================] - 6s 643us/sample - loss: 0.0039 - mae: 0.0489 

In [12]:
history_dnn = model_dnn.train(train_ts, normed_thetas, batch_size=256, 
                      epochs=500, verbose=1, learning_rate=0.001, 
                      early_stopping_patience=5,
                      validation_inputs=validation_ts, validation_targets=normed_thetas_val)

Train on 10000 samples, validate on 10000 samples
Epoch 1/500
10000/10000 [==============================] - 3s 338us/sample - loss: 0.9258 - mae: 0.7184 - val_loss: 0.1055 - val_mae: 0.2595
Epoch 2/500
10000/10000 [==============================] - 1s 59us/sample - loss: 0.1157 - mae: 0.2646 - val_loss: 0.1067 - val_mae: 0.2628
Epoch 3/500
10000/10000 [==============================] - 1s 63us/sample - loss: 0.0739 - mae: 0.2129 - val_loss: 0.0767 - val_mae: 0.2185
Epoch 4/500
10000/10000 [==============================] - 1s 80us/sample - loss: 0.0552 - mae: 0.1846 - val_loss: 0.0736 - val_mae: 0.2128
Epoch 5/500
10000/10000 [==============================] - 1s 65us/sample - loss: 0.0446 - mae: 0.1666 - val_loss: 0.0702 - val_mae: 0.2069
Epoch 6/500
10000/10000 [==============================] - 1s 74us/sample - loss: 0.0374 - mae: 0.1516 - val_loss: 0.0629 - val_mae: 0.1942
Epoch 7/500
10000/10000 [==============================] - 1s 71us/sample - loss: 0.0317 - mae: 0.1400 - val_

10000/10000 [==============================] - 1s 99us/sample - loss: 0.0060 - mae: 0.0602 - val_loss: 0.0219 - val_mae: 0.1103
Epoch 59/500
10000/10000 [==============================] - 1s 82us/sample - loss: 0.0053 - mae: 0.0569 - val_loss: 0.0213 - val_mae: 0.1086
Epoch 60/500
10000/10000 [==============================] - 1s 87us/sample - loss: 0.0052 - mae: 0.0564 - val_loss: 0.0213 - val_mae: 0.1091
Epoch 61/500
10000/10000 [==============================] - 1s 97us/sample - loss: 0.0054 - mae: 0.0570 - val_loss: 0.0211 - val_mae: 0.1082
Epoch 62/500
10000/10000 [==============================] - 1s 121us/sample - loss: 0.0050 - mae: 0.0549 - val_loss: 0.0207 - val_mae: 0.1075
Epoch 63/500
10000/10000 [==============================] - 1s 88us/sample - loss: 0.0049 - mae: 0.0550 - val_loss: 0.0209 - val_mae: 0.1079
Epoch 64/500
10000/10000 [==============================] - 1s 104us/sample - loss: 0.0049 - mae: 0.0545 - val_loss: 0.0203 - val_mae: 0.1063
Epoch 65/500
10000/10000

Epoch 116/500
10000/10000 [==============================] - 1s 94us/sample - loss: 0.0032 - mae: 0.0438 - val_loss: 0.0145 - val_mae: 0.0904
Epoch 117/500
10000/10000 [==============================] - 1s 103us/sample - loss: 0.0028 - mae: 0.0414 - val_loss: 0.0145 - val_mae: 0.0906
Epoch 118/500
10000/10000 [==============================] - 1s 86us/sample - loss: 0.0030 - mae: 0.0428 - val_loss: 0.0144 - val_mae: 0.0900
Epoch 119/500
10000/10000 [==============================] - 1s 83us/sample - loss: 0.0029 - mae: 0.0423 - val_loss: 0.0147 - val_mae: 0.0914
Epoch 120/500
10000/10000 [==============================] - 1s 85us/sample - loss: 0.0027 - mae: 0.0408 - val_loss: 0.0141 - val_mae: 0.0895
Epoch 121/500
10000/10000 [==============================] - 1s 84us/sample - loss: 0.0030 - mae: 0.0422 - val_loss: 0.0141 - val_mae: 0.0893
Epoch 122/500
10000/10000 [==============================] - 1s 84us/sample - loss: 0.0028 - mae: 0.0417 - val_loss: 0.0139 - val_mae: 0.0886
Epoch

In [13]:
history_pen = model_pen.train(train_ts, normed_thetas, batch_size=256, 
                      epochs=500, verbose=1, learning_rate=0.001, 
                      early_stopping_patience=5,
                      validation_inputs=validation_ts, validation_targets=normed_thetas_val)

Train on 10000 samples, validate on 10000 samples
Epoch 1/500
10000/10000 [==============================] - 9s 935us/sample - loss: 0.2161 - mae: 0.3377 - val_loss: 0.1006 - val_mae: 0.2792
Epoch 2/500
10000/10000 [==============================] - 4s 382us/sample - loss: 0.0515 - mae: 0.1767 - val_loss: 0.0735 - val_mae: 0.2340
Epoch 3/500
10000/10000 [==============================] - 4s 419us/sample - loss: 0.0342 - mae: 0.1451 - val_loss: 0.0495 - val_mae: 0.1857
Epoch 4/500
10000/10000 [==============================] - 4s 401us/sample - loss: 0.0278 - mae: 0.1307 - val_loss: 0.0334 - val_mae: 0.1466
Epoch 5/500
10000/10000 [==============================] - 3s 347us/sample - loss: 0.0226 - mae: 0.1178 - val_loss: 0.0245 - val_mae: 0.1243
Epoch 6/500
10000/10000 [==============================] - 4s 366us/sample - loss: 0.0187 - mae: 0.1072 - val_loss: 0.0228 - val_mae: 0.1200
Epoch 7/500
10000/10000 [==============================] - 4s 364us/sample - loss: 0.0169 - mae: 0.1019 

10000/10000 [==============================] - 5s 516us/sample - loss: 0.0028 - mae: 0.0417 - val_loss: 0.0041 - val_mae: 0.0496
Epoch 59/500
10000/10000 [==============================] - 5s 517us/sample - loss: 0.0028 - mae: 0.0414 - val_loss: 0.0039 - val_mae: 0.0482
Epoch 60/500
10000/10000 [==============================] - 4s 360us/sample - loss: 0.0027 - mae: 0.0403 - val_loss: 0.0038 - val_mae: 0.0476
Epoch 61/500
10000/10000 [==============================] - 4s 442us/sample - loss: 0.0028 - mae: 0.0416 - val_loss: 0.0039 - val_mae: 0.0483
Epoch 62/500
10000/10000 [==============================] - 5s 460us/sample - loss: 0.0026 - mae: 0.0397 - val_loss: 0.0036 - val_mae: 0.0461
Epoch 63/500
10000/10000 [==============================] - 4s 444us/sample - loss: 0.0025 - mae: 0.0393 - val_loss: 0.0035 - val_mae: 0.0457
Epoch 64/500
10000/10000 [==============================] - 5s 510us/sample - loss: 0.0026 - mae: 0.0400 - val_loss: 0.0039 - val_mae: 0.0484
Epoch 65/500
10000/

In [14]:
# Routine to test each xNN architecture on MAE
from sklearn.metrics import mean_absolute_error
def test_model(model):
    pred_test = model.predict(test_ts)
    pred_test = denormalize_data(pred_test, dmin, dmax)
    #mae_test = np.mean(abs(pred_test - samples_test), axis=0)
    mae_test = mean_absolute_error(test_thetas, pred_test)
    
    theta_pred = model.predict(data)
    samples_true = np.asarray(true_param)
    theta_pred = denormalize_data(theta_pred, dmin, dmax)
    mae_true = np.mean(np.abs(theta_pred.ravel() - samples_true), axis=0)
    #mae_true = mean_absolute_error(np.asarray(samples_true).reshape(1,2), np.asarray(theta_pred).reshape(1,2))
    return mae_test, mae_true

In [15]:
# Calculate test metrics
mae_test_cnn, mae_true_cnn = test_model(model_cnn)
mae_test_dnn, mae_true_dnn = test_model(model_dnn)
mae_test_pen, mae_true_pen = test_model(model_pen)

In [16]:
print("CNN test MAE = {}, MAE at true point = {}".format(mae_test_cnn, mae_true_cnn))
print("DNN test MAE = {}, MAE at true point = {}".format(mae_test_dnn, mae_true_dnn))
print("PEN test MAE = {}, MAE at true point = {}".format(mae_test_pen, mae_true_pen))

CNN test MAE = 0.6971441133165779, MAE at true point = 0.032827445864677424
DNN test MAE = 0.7202301375520954, MAE at true point = 0.17880656123161318
PEN test MAE = 0.7494010777132241, MAE at true point = 0.10754189789295196


So here we used the xNN models to directly infer the parameters from given time series. The xNN models can also be used as summary statistics in conjunction with ABC inference. The following Class implements one such summary statistic.

In [53]:
from sciope.utilities.summarystats.summary_base import SummaryBase
from sciope.utilities.housekeeping import sciope_logger as ml

class ANN_Statistics(SummaryBase):
    """
    The thetas predicted by xNN models act as summary statistics
    """

    def __init__(self, mean_trajectories=False, use_logger=False):
        self.name = 'ANN_Statistics'
        super(ANN_Statistics, self).__init__(self.name, mean_trajectories, use_logger)
        if self.use_logger:
            self.logger = ml.SciopeLogger().get_logger()
            self.logger.info("ANN_Statistics summary statistic initialized")

    def compute(self, data):
        """
        Calculate the value(s) of the summary statistic(s)
        
        Parameters
        ----------
        data : [type]
            simulated or data set in the form N x S X T - num data points x num species x num time steps
        
        Returns
        -------
        [type]
            computed statistic value
        
        """
        data_arr = np.array(data)
        assert len(data_arr.shape) == 3, "required input shape is (n_points, n_species, n_timepoints)"

        res = model_cnn.predict(data_arr)
        res = denormalize_data(res, dmin, dmax)

        if self.mean_trajectories:
            res = np.asarray(np.mean(res, axis=0))  # returns a scalar, so we cast it as an array

        if self.use_logger:
            self.logger.info("ANN_Statistics summary statistic: processed data matrix of shape {0} and generated summaries"
                             " of shape {1}".format(data.shape, res.shape))
        return res

In [54]:
# Test our new summary statistic
cnn_stat = ANN_Statistics()
predicted_stat = cnn_stat.compute(data)

In [19]:
print(predicted_stat)
stat_mae = np.mean(np.abs(predicted_stat.ravel() - np.asarray(true_param)), axis=0)
print("MAE upon comparison as a statistic = {}".format(stat_mae))

[[0.5757128  0.15863231]]
MAE upon comparison as a statistic = 0.032827445864677424


Now we are ready to set up ABC inference. Here we use Burstiness as a summary statistic. Auto-covariance is more appropriate for MA(2). We can use our xNN model as well, but it should be trained on far more points in order to be accurate enough for use in inference. This notebook is meant to be quick to execute, and so we have used a small training set for exposition.

In [20]:
# The MA2 simulator function
def ma2_sim(param):
    res = np.expand_dims(np.array([simulate(param, n=100)]), 2)
    res = res.transpose((0, 2, 1))    # reshape to N x S X T
    return res

In [30]:
# Generate observed data
obs_data = []
for i in range(20):
    obs_data.append(ma2_sim(true_param))
obs_data = np.asarray(obs_data)
obs_data = obs_data[:, 0, :, :]

In [68]:
from sciope.inference import abc_inference
from sciope.utilities.priors import uniform_prior
from sciope.utilities.summarystats import burstiness

In [73]:
prior_function = uniform_prior.UniformPrior(dmin, dmax)
bs_stat = burstiness.Burstiness()
abc_instance = abc_inference.ABC(obs_data, ma2_sim, prior_function, epsilon=0.01, 
                                 #summaries_function=cnn_stat.compute, # ensure xNN is accurate enough before use
                                 summaries_function=bs_stat.compute,
                                 summaries_divisor=np.max(abc_trial_thetas, axis=0), use_logger=True)


2019-12-20 00:40:27,127 	 [INFO     | abc_inference.py:91] : Approximate Bayesian Computation initialized


In [74]:
abc_instance.compute_fixed_mean(chunk_size=2)

In [75]:
abc_results = abc_instance.infer(num_samples=50, batch_size=10, chunk_size=2)


2019-12-20 00:40:28,837 	 [INFO     | abc_inference.py:257] : running in parallel mode

2019-12-20 00:40:28,927 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [-0.45665561 -0.81897324]

2019-12-20 00:40:28,930 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.02573464 0.00815543]

2019-12-20 00:40:28,932 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [ 0.18864251 -0.11024079]

2019-12-20 00:40:28,937 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.13674358 0.3781142 ]

2019-12-20 00:40:28,940 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [2.89364722 1.99037204]

2019-12-20 00:40:28,942 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.02276618 0.24063794]

2019-12-20 00:40:28,947 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [0.96552766


2019-12-20 00:40:29,782 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [ 2.34716991 -0.94728343]

2019-12-20 00:40:29,793 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.29715093 0.31133826]

2019-12-20 00:40:29,799 	 [INFO     | abc_inference.py:257] : running in parallel mode

2019-12-20 00:40:29,877 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [-0.52668802 -0.03880795]

2019-12-20 00:40:29,898 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.00373382 0.16530002]

2019-12-20 00:40:29,902 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [-0.00985655 -0.50877993]

2019-12-20 00:40:29,912 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.03632841 0.29605257]

2019-12-20 00:40:29,918 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [-0.81351


2019-12-20 00:40:30,622 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [1.28380653 1.45410007]

2019-12-20 00:40:30,628 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.00043619 0.05870903]

2019-12-20 00:40:30,631 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [1.38849847 0.48167356]

2019-12-20 00:40:30,635 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.07124408 0.15711193]

2019-12-20 00:40:30,639 	 [INFO     | abc_inference.py:257] : running in parallel mode

2019-12-20 00:40:30,707 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [ 2.43769595 -0.42360618]

2019-12-20 00:40:30,712 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.05257271 0.3032262 ]

2019-12-20 00:40:30,719 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [3.18026632 1


2019-12-20 00:40:31,321 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [3.50296784 1.75134086]

2019-12-20 00:40:31,325 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.02722307 0.42247622]

2019-12-20 00:40:31,329 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [2.96808911 0.20631696]

2019-12-20 00:40:31,336 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.04081487 0.06951561]

2019-12-20 00:40:31,357 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [3.78916428 0.75070262]

2019-12-20 00:40:31,362 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.142948   0.01907407]

2019-12-20 00:40:31,366 	 [INFO     | abc_inference.py:257] : running in parallel mode

2019-12-20 00:40:31,426 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [1.76331563 1.7


2019-12-20 00:40:32,066 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.15601302 0.17616268]

2019-12-20 00:40:32,070 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [3.67460297 0.4487887 ]

2019-12-20 00:40:32,081 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.03736246 0.12906502]

2019-12-20 00:40:32,085 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [3.5702724  0.89442778]

2019-12-20 00:40:32,100 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.23878455 0.15906772]

2019-12-20 00:40:32,102 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [0.06549889 0.46673703]

2019-12-20 00:40:32,106 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.05576989 0.00388545]

2019-12-20 00:40:32,118 	 [INFO     | abc_inference.py:257] : running in parall

...


2019-12-20 00:48:54,952 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.07461697 0.30894656]

2019-12-20 00:48:54,959 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [ 1.00119939 -0.25497162]

2019-12-20 00:48:54,968 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.04476444 0.23041205]

2019-12-20 00:48:54,972 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [ 3.48216032 -0.18827226]

2019-12-20 00:48:54,987 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.0139183  0.43545684]

2019-12-20 00:48:54,992 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [3.35636108 1.67827114]

2019-12-20 00:48:55,004 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.12589648 0.21615167]

2019-12-20 00:48:55,026 	 [DEBUG    | abc_inference.py:297] : ABC Rejection


2019-12-20 00:48:56,254 	 [INFO     | abc_inference.py:257] : running in parallel mode

2019-12-20 00:48:56,403 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [-1.22466175  1.25721194]

2019-12-20 00:48:56,406 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.01854718 0.37025691]

2019-12-20 00:48:56,410 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [0.53707663 1.12290002]

2019-12-20 00:48:56,419 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.0479429  0.13874376]

2019-12-20 00:48:56,425 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [ 1.48948038 -0.87044827]

2019-12-20 00:48:56,435 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.0538446  0.00317968]

2019-12-20 00:48:56,441 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [-0.9121128


2019-12-20 00:48:57,651 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.04561402 0.02464043]

2019-12-20 00:48:57,663 	 [INFO     | abc_inference.py:257] : running in parallel mode

2019-12-20 00:48:57,823 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [0.96724225 0.69198845]

2019-12-20 00:48:57,829 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.02732753 0.23350122]

2019-12-20 00:48:57,835 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [ 1.61465233 -0.50102271]

2019-12-20 00:48:57,839 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.03647825 0.68825261]

2019-12-20 00:48:57,843 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [3.75891385 1.00442832]

2019-12-20 00:48:57,849 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.25689377 0.


2019-12-20 00:48:58,991 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.4271178  0.03358438]

2019-12-20 00:48:59,029 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [-1.95728491 -0.95163486]

2019-12-20 00:48:59,039 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.07313864 0.16631528]

2019-12-20 00:48:59,058 	 [INFO     | abc_inference.py:257] : running in parallel mode

2019-12-20 00:48:59,188 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [1.96656667 0.91485004]

2019-12-20 00:48:59,190 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.00536023 0.01759362]

2019-12-20 00:48:59,199 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [0.28334552 1.48762767]

2019-12-20 00:48:59,232 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.03835268 0.


2019-12-20 00:49:00,396 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [ 2.84995185 -0.93304707]

2019-12-20 00:49:00,400 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.09378738 0.1529366 ]

2019-12-20 00:49:00,414 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [2.95108185 0.53002606]

2019-12-20 00:49:00,430 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.20101519 0.21910494]

2019-12-20 00:49:00,432 	 [INFO     | abc_inference.py:257] : running in parallel mode

2019-12-20 00:49:00,607 	 [DEBUG    | abc_inference.py:297] : ABC Rejection Sampling: trial parameter(s) = [3.56404867 1.21756839]

2019-12-20 00:49:00,610 	 [DEBUG    | abc_inference.py:298] : ABC Rejection Sampling: trial distance(s) = [0.00378473 0.00053062]

2019-12-20 00:49:00,619 	 [INFO     | abc_inference.py:305] : ABC Rejection Sampling: accepted a new sample, total accep

In [80]:
# Calculate MAE
acc_samples = np.asarray(abc_results['accepted_samples'])
abc_mae = np.mean(np.abs(acc_samples - np.asarray(true_param).reshape(1, 2)), axis=0)
abc_mae = np.mean(abc_mae)   # mean of both thetas
print("ABC mean inference error = {}".format(abc_mae))

ABC mean inference error = 1.1917190758994305
